In [1]:
%run scrapeStats.ipynb
%run scrapeSpreads.ipynb
%run scrapeFO.ipynb
import statistics
import numpy as np


Getting a list of all teams.
Done!
Building two lists of urls to parse
Done!
Getting stats from teamrankings.com


In [2]:
import matplotlib.pyplot as plt

In [3]:
combined = pd.DataFrame(pd.merge(FO(), team_stats(), left_on="Long",right_on="teamname"))
fullstats = combined.iloc[:,[0,1,2,3,5,6,7,8]]


In [4]:
fullstats['yppAdj'] = (1+(fullstats['Offense DVOA']/100)) *  fullstats['yppcol']
fullstats['oYPPAdj'] = (1+(fullstats['Defense DVOA']/100)) * fullstats['oYPPcol']

In [5]:
homedf = pd.merge(fullstats, spreads(), left_on ="Long", right_on = "homeTeam")
awaydf = pd.merge(fullstats, spreads(), left_on ="Long", right_on = "awayTeam")

In [6]:
fulljoin = pd.merge(homedf, awaydf, left_on = "homeTeam", right_on = "homeTeam")

In [7]:
fulljoin['choasRate_x'] = (fulljoin['qbsackedcol_y'] + fulljoin['sackpctcol_x']) / 2
fulljoin['choasRate_y'] = (fulljoin['qbsackedcol_x'] + fulljoin['sackpctcol_y']) / 2
fulljoin['choasRate_ratio'] = (fulljoin['choasRate_x'] / 10) - (fulljoin['choasRate_y'] / 10)
fulljoin['yppavg'] = (fulljoin['yppAdj_x'] + fulljoin['oYPPAdj_y']) / 2
fulljoin['oYPPavg'] = (fulljoin['oYPPAdj_x'] + fulljoin['yppAdj_y']) / 2
fulljoin['netYPP'] = (fulljoin['yppavg'] - fulljoin['oYPPavg']) + fulljoin['choasRate_ratio']

In [8]:
x = np.array([0, 0.5, 1, 1.5, 2, 3, 4, 5, 6])
y = np.array([0, -0.65, -4.25, -5.5, -10.425, -15.075, -19.15, -24.18, -29.66])
z = np.polyfit(x,y,1)
predict = np.poly1d(z)

fulljoin['proj_Homespread'] = predict(fulljoin['netYPP'])

final = fulljoin[['Long_x','Long_y','homeSpread_x','proj_Homespread']]
final.columns = ['home','away','homeSpread','projSpread']
final['homeSpread'] = final['homeSpread'].astype(float)
final['diff'] = abs(final['homeSpread'] - final['projSpread'])

C:\Users\forlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\forlo\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
final.sort_values(by=['diff'], ascending=False)

,home,away,homeSpread,projSpread,diff
3,Green Bay Packers,Atlanta Falcons,-16.5,-6.307731,10.192269
2,Baltimore Ravens,Cincinnati Bengals,-13.5,-4.817033,8.682967
7,Houston Texans,Jacksonville Jaguars,-6.0,1.113557,7.113557
6,Cleveland Browns,Indianapolis Colts,3.0,7.915897,4.915897
0,Pittsburgh Steelers,Philadelphia Eagles,-7.5,-12.170873,4.670873
1,Seattle Seahawks,Minnesota Vikings,-7.0,-3.694508,3.305492
8,New York Jets,Arizona Cardinals,7.0,9.406595,2.406595
5,Chicago Bears,Tampa Bay Buccaneers,6.0,7.960778,1.960778
4,Dallas Cowboys,New York Giants,-9.5,-8.519309,0.980691
